In [45]:
#just changing to BNS prior dict. keeping geocentric time, and reoving geocentric time 

In [46]:
# hello

In [1]:
import sys
sys.path.insert(0, "/home/marc.penuliar/bilby")
import bilby
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

No version information file '.version' found


In [48]:
# import bilby
from gwpy.timeseries import TimeSeries

logger = bilby.core.utils.logger
outdir = "outdir"
label = "GW190425"


In [49]:
# Note you can get trigger times using the gwosc package, e.g.
# > from gwosc import datasets
# > datasets.event_gps("GW190425")
trigger_time = 1240215503.0
detectors = ["L1", "V1"]
maximum_frequency = 512
minimum_frequency = 20
roll_off = 0.4  # Roll off duration of tukey window in seconds, default is 0.4s
duration = 128  # Analysis segment duration
post_trigger_duration = 2  # Time between trigger time and end of segment
end_time = trigger_time + post_trigger_duration
start_time = end_time - duration


In [50]:
# The fiducial parameters are taken to me the max likelihood sample from the
# posterior sample release of LIGO-Virgo
# https://www.gw-openscience.org/eventapi/html/O3_Discovery_Papers/GW190425/
# The fiducial parameters should always be in provided in the same basis as
# the sampling basis. For example, if sampling in  `mass_1` and `mass_2` instead of
# `chirp_mass` and `mass_ratio`, the fiducial parameters should also be provided in
# `mass_1` and `mass_2` below.

fiducial_parameters = {
    "a_1": 0.018, #Had to comment these out since we had other priors. Changed, had to uncomment this and comment out the priros of a_1 and a_2? 
    "a_2": 0.016,
    "chirp_mass": 1.48658,
    "dec": 0.438,
    "geocent_time": 1240215503.039,
    "lambda_1": 446.941,
    "lambda_2": 43.386,
    "luminosity_distance": 206.751,
    "mass_ratio": 0.8955,
    "phase": 3.0136566567608765,
    "phi_12": 4.319,
    "phi_jl": 5.07,
    "psi": 0.281,
    "ra": 4.2,
    "theta_jn": 0.185,
    "tilt_1": 0.879,
    "tilt_2": 0.514,
}
psd_duration = 1024
psd_start_time = start_time - psd_duration
psd_end_time = start_time


In [51]:
# We now use gwpy to obtain analysis and psd data and create the ifo_list
ifo_list = bilby.gw.detector.InterferometerList([])
for det in detectors:
    logger.info("Downloading analysis data for ifo {}".format(det))
    ifo = bilby.gw.detector.get_empty_interferometer(det)
    data = TimeSeries.fetch_open_data(det, start_time, end_time)
    ifo.strain_data.set_from_gwpy_timeseries(data)

    logger.info("Downloading psd data for ifo {}".format(det))
    psd_data = TimeSeries.fetch_open_data(det, psd_start_time, psd_end_time)
    psd_alpha = 2 * roll_off / duration
    psd = psd_data.psd(
        fftlength=duration, overlap=0, window=("tukey", psd_alpha), method="median"
    )
    ifo.power_spectral_density = bilby.gw.detector.PowerSpectralDensity(
        frequency_array=psd.frequencies.value, psd_array=psd.value
    )
    ifo.maximum_frequency = maximum_frequency
    ifo.minimum_frequency = minimum_frequency
    ifo_list.append(ifo)

logger.info("Saving data plots to {}".format(outdir))
bilby.core.utils.check_directory_exists_and_if_not_mkdir(outdir)
ifo_list.plot_data(outdir=outdir, label=label)


00:17 bilby INFO    : Downloading analysis data for ifo L1
00:17 bilby INFO    : Downloading psd data for ifo L1
00:17 bilby INFO    : Downloading analysis data for ifo V1
00:17 bilby INFO    : Downloading psd data for ifo V1
00:17 bilby INFO    : Saving data plots to outdir
00:17 bilby INFO    : Generating frequency domain strain from given time domain strain.
00:17 bilby INFO    : Applying a tukey window with alpha=0.003125, roll off=0.2
00:17 bilby INFO    : Generating frequency domain strain from given time domain strain.
00:17 bilby INFO    : Applying a tukey window with alpha=0.003125, roll off=0.2


In [52]:
# We now define the prior.
# We have defined our prior distribution in a local file, GW190425.prior
# The prior is printed to the terminal at run-time.
# You can overwrite this using the syntax below in the file,
# or choose a fixed value by just providing a float value as the prior.
priors = bilby.gw.prior.BNSPriorDict(filename="GW190425.prior")
for key in [
    "psi",
    "geocent_time",
    "ra",
    "dec",
    # "chi_1", #since I had to comment out the a_1 and a_2 due to errors from results i had to uncomment chi_1 and chi_2 back in 
    # "chi_2",
    "theta_jn",
    # "luminosity_distance", #ValueError: Cannot use marginalized likelihood for phase: prior is fixed
    # "phase",
]:
    priors[key] = fiducial_parameters[key]
del (
    priors["lambda_1"],
    priors["lambda_2"],
    priors["mass_ratio"],
)


priors["chirp_mass"] = bilby.core.prior.Gaussian(
    1.215, 0.1, name="chirp_mass", unit="$M_{\\odot}$"
)
priors["symmetric_mass_ratio"] = bilby.core.prior.Uniform(
    0.1, 0.25, name="symmetric_mass_ratio"
)

# Using this prior will create lambda1 and lambda2. Outputs are funky
priors["lambda_symmetric"] = bilby.core.prior.Uniform(0, 5000, name="lambda_symmetric")

priors["binary_love_uniform"] = bilby.core.prior.Uniform(
    0, 1, name="binary_love_uniform"
)

priors["lambda_1"] = bilby.core.prior.Constraint(
    name="lambda_1", minimum=0, maximum=10000
)
priors["lambda_2"] = bilby.core.prior.Constraint(
    name="lambda_2", minimum=0, maximum=10000
)
##Going to comment this out because ValueError: Cannot use marginalized likelihood for phase: prior is fixed

priors["phase"] = bilby.core.prior.Uniform(
    0, 2 * np.pi, boundary="periodic", name="phase"
)


# #Had to comment this out. Since i got errors with results variable
# priors["a_1"] = bilby.core.prior.Constraint(
#     name="a_1", minimum=0, maximum=0.05
# )
# priors["a_2"] = bilby.core.prior.Constraint(
#     name="a_2", minimum=0, maximum=0.05
# )

# Add the geocent time prior
priors["geocent_time"] = bilby.core.prior.Uniform(
    trigger_time - 0.1, trigger_time + 0.1, name="geocent_time"
)

In [53]:
# In this step we define a `waveform_generator`. This is the object which
# creates the frequency-domain strain. In this instance, we are using the
# `lal_binary_black_hole model` source model. We also pass other parameters:
# the waveform approximant and reference frequency and a parameter conversion
# which allows us to sample in chirp mass and ratio rather than component mass

# Set the duration and sampling frequency of the data segment that we're going
# to inject the signal into. For the
# TaylorF2 waveform, we cut the signal close to the isco frequency
duration = 32
sampling_frequency = 4096  # 2048 #4096 changed for later. we need faster test runs

#Removed since it was used previously in a cell above 
# start_time = injection_parameters["geocent_time"] + 2 - duration 


# Fixed arguments passed into the source model. The analysis starts at 40 Hz.
waveform_arguments = dict(
    waveform_approximant="IMRPhenomPv2_NRTidal",  # IMRPhenomPv2_NRTidal
    reference_frequency=50.0,
    minimum_frequency=40.0,
)


# Create the waveform_generator using a LAL Binary Neutron Star source function
waveform_generator = bilby.gw.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    frequency_domain_source_model=bilby.gw.source.lal_binary_neutron_star,
    parameter_conversion=bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters,
    waveform_arguments=waveform_arguments,
)


00:17 bilby INFO    : Waveform generator initiated with
  frequency_domain_source_model: bilby.gw.source.lal_binary_neutron_star
  time_domain_source_model: None
  parameter_conversion: bilby.gw.conversion.convert_to_lal_binary_neutron_star_parameters


In [54]:
# In this step, we define the likelihood. Here we use the standard likelihood
# function, passing it the data and the waveform generator.
# Note, phase_marginalization is formally invalid with a precessing waveform such as IMRPhenomPv2
# likelihood = bilby.gw.likelihood.RelativeBinningGravitationalWaveTransient(
#     ifo_list,
#     waveform_generator,
#     priors=priors,
#     time_marginalization=False,
#     phase_marginalization=True,
#     distance_marginalization=True,
#     # fiducial_parameters=fiducial_parameters,
# )


likelihood = bilby.gw.GravitationalWaveTransient(
    ifo_list,
    waveform_generator,
    priors=priors,
    time_marginalization=False,
    phase_marginalization=True,
    distance_marginalization=True,
    # fiducial_parameters=fiducial_parameters,
)


# likelihood = bilby.gw.GravitationalWaveTransient(
#     interferometers=detectors,
#     waveform_generator=waveform_generator,
#     phase_marginalization=True,
#     priors=priors,
# )

00:17 bilby INFO    : Loaded distance marginalisation lookup table from .distance_marginalization_lookup.npz.


In [57]:
# Finally, we run the sampler. This function takes the likelihood and prior
# along with some options for how to do the sampling and how to save the data
result = bilby.run_sampler(
    likelihood,
    priors,
    sampler="dynesty", #Orignally i was using nestler 
    outdir=outdir,
    label=label,
    nlive=100, #changed from 1000 to 100 to see if i save on time? Been having errors late in the run and i want to see them sooner 
    check_point_plot=True,
    npool=1,
    conversion_function=bilby.gw.conversion.generate_all_bns_parameters,
)
result.plot_corner()


00:24 bilby INFO    : Running for label 'GW190425', output will be saved to 'outdir'
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24 bilby INFO    : Performing redundancy check using BBHPriorDict(self).test_redundancy
00:24

0it [00:00, ?it/s]

00:25 bilby INFO    : Using sampler Dynesty with kwargs {'bound': 'multi', 'sample': 'rwalk', 'print_progress': True, 'periodic': None, 'reflective': None, 'check_point_delta_t': 1800, 'nlive': 100, 'first_update': None, 'walks': 100, 'npdim': None, 'rstate': None, 'queue_size': 1, 'pool': None, 'use_pool': None, 'live_points': None, 'logl_args': None, 'logl_kwargs': None, 'ptform_args': None, 'ptform_kwargs': None, 'enlarge': 1.5, 'bootstrap': None, 'vol_dec': 0.5, 'vol_check': 8.0, 'facc': 0.2, 'slices': 5, 'update_interval': 60, 'print_func': <bound method Dynesty._print_func of <bilby.core.sampler.dynesty.Dynesty object at 0x7fdf35ceb760>>, 'dlogz': 0.1, 'maxiter': None, 'maxcall': None, 'logl_max': inf, 'add_live': True, 'save_bounds': False, 'n_effective': None, 'maxmcmc': 5000, 'nact': 5, 'print_method': 'tqdm'}
00:25 bilby INFO    : Checkpoint every check_point_delta_t = 600s
00:25 bilby INFO    : Using dynesty version 2.1.0
00:25 bilby INFO    : Using the bilby-implemented rwa

No version information file '.version' found


TypeError: __new__() got an unexpected keyword argument 'check_point_delta_t'

In [ ]:
# This is confusing, since lambda1 and lambda2 does not appear in the posterior samples. But does plot.
# The next cell proves that lambda1 and lambda2 are not in the results.posterior samples.
result.posterior['lambda1'] = result.posterior.apply(lambda row :
        bilby.gw.conversion.binary_love_lambda_symmetric_to_lambda_1_lambda_2_manual_marginalisation(
                row['binary_love_uniform'],row['lambda_symmetric'],row['symmetric_mass_ratio']
                )[0], axis = 1)

result.posterior['lambda2'] = result.posterior.apply(lambda row :
        bilby.gw.conversion.binary_love_lambda_symmetric_to_lambda_1_lambda_2_manual_marginalisation(
                row['binary_love_uniform'],row['lambda_symmetric'],row['symmetric_mass_ratio']
                )[1], axis = 1)

sns.kdeplot(
    x=result.posterior["lambda_1"],
    y=result.posterior["lambda_2"],
    color="green",
    label="Binary_love_&_LambdaSymm",
)

####
# title
plt.title("Universal Realtions Accuracy")

# lambda 2 y
plt.axhline(y=result.injection_parameters["lambda_2"], color="orange", linestyle="-")
# lambda 2 x
plt.axvline(x=result.injection_parameters["lambda_1"], color="orange")
# Add the point
plt.scatter(
    result.injection_parameters["lambda_1"],
    result.injection_parameters["lambda_2"],
    color="orange",
)

plt.show()

In [ ]:
# print(result.posterior)

In [ ]:
#    check_point_delta_t=600,